*Think Linear Algebra* is not for sale yet, but if you would like to support this project, you can [buy me a coffee](https://buymeacoffee.com/allendowney).

# Trusses

[Click here to run this notebook on Colab](https://colab.research.google.com/github/AllenDowney/ThinkLinearAlgebra/blob/main/nb/nullspace.ipynb).

In [49]:
from os.path import basename, exists


def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve

        local, _ = urlretrieve(url, filename)
        print("Downloaded " + local)


download("https://github.com/AllenDowney/ThinkLinearAlgebra/raw/main/utils.py")

In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sympy as sp

from utils import decorate

## Section

In [51]:
# --- Imports ---
import pint
from sympy import symbols, Matrix, sqrt, simplify, Eq, solve

# --- Unit setup ---
ureg = pint.UnitRegistry()

# --- Symbolic variables (global scope) ---
L = symbols('L')
x_C, y_C = symbols('x_C y_C')
E_CA, E_CB = symbols('E_CA E_CB')
A_CA, A_CB = symbols('A_CA A_CB')
F = symbols('F')
u_x, u_y = symbols('u_x u_y')

# --- Position vectors of nodes ---
pos_A = Matrix([0, 0])
pos_B = Matrix([L, 0])
pos_C = Matrix([x_C, y_C])

# --- Displacement vector at node C ---
u_C = Matrix([u_x, u_y])
u_C

Matrix([
[u_x],
[u_y]])

In [70]:
# --- Vectors from node C to A and B ---
r_CA = pos_A - pos_C
r_CB = pos_B - pos_C

# --- Stiffnesses (scalar) ---
L_CA = r_CA.norm()
L_CB = r_CB.norm()

k_CA = E_CA * A_CA / L_CA
k_CB = E_CB * A_CB / L_CB
k_CA

A_CA*E_CA/sqrt(Abs(x_C)**2 + Abs(y_C)**2)

In [74]:
def projection(r, v):
    rhat = r.normalized()
    return rhat.dot(v) * rhat


# --- Internal force contributions ---
f_CA = -k_CA * projection(r_CA, u_C)
f_CB = -k_CB * projection(r_CB, u_C)

f_C = f_CA + f_CB  # Net internal force on node C
f_C

Matrix([
[A_CA*E_CA*x_C*(-u_x*x_C/sqrt(Abs(x_C)**2 + Abs(y_C)**2) - u_y*y_C/sqrt(Abs(x_C)**2 + Abs(y_C)**2))/(Abs(x_C)**2 + Abs(y_C)**2) - A_CB*E_CB*(L - x_C)*(u_x*(L - x_C)/sqrt(Abs(y_C)**2 + Abs(L - x_C)**2) - u_y*y_C/sqrt(Abs(y_C)**2 + Abs(L - x_C)**2))/(Abs(y_C)**2 + Abs(L - x_C)**2)],
[      A_CA*E_CA*y_C*(-u_x*x_C/sqrt(Abs(x_C)**2 + Abs(y_C)**2) - u_y*y_C/sqrt(Abs(x_C)**2 + Abs(y_C)**2))/(Abs(x_C)**2 + Abs(y_C)**2) + A_CB*E_CB*y_C*(u_x*(L - x_C)/sqrt(Abs(y_C)**2 + Abs(L - x_C)**2) - u_y*y_C/sqrt(Abs(y_C)**2 + Abs(L - x_C)**2))/(Abs(y_C)**2 + Abs(L - x_C)**2)]])

In [75]:
# --- Numerical parameters (Pint quantities) ---
params = {
    'L': 15.0 * ureg.cm,
    'x_C': 15.0 / 2 * ureg.cm,
    'y_C': np.sqrt(3) * 15.0 / 2 * ureg.cm,
    'A_CA': 1.0 * ureg.cm * 0.2 * ureg.cm,
    'A_CB': 1.0 * ureg.cm * 0.2 * ureg.cm,
    'E_CA': 3.0e9 * ureg.Pa,
    'E_CB': 3.0e9 * ureg.Pa,
    'F': 9.8 * ureg.newton,
}

# --- Helper: map symbols to numerical values ---
def make_subs(symbols, quantities):
    return {sym: quantities[sym.name].to_base_units().magnitude for sym in symbols}

symbols_to_sub = [L, x_C, y_C, A_CA, A_CB, E_CA, E_CB, F]
subs = make_subs(symbols_to_sub, params)

# --- External force vector ---
f_ext = Matrix([0, -F])  # Downward load

# --- Equilibrium equations (sum of forces = 0) ---
eqs = [
    Eq(f_C[0] + f_ext[0], 0),
    Eq(f_C[1] + f_ext[1], 0)
]

# --- Solve for displacement under load ---
eqs_num = [eq.subs(subs) for eq in eqs]
sol = solve(eqs_num, (u_x, u_y), dict=True)

# --- Display results in mm ---
print("Displacement at node C:")
for k, v in sol[0].items():
    print(f"{k} = {v*1000} mm")

Displacement at node C:
u_x = 0 mm
u_y = -0.0163333333333333 mm


[Think Linear Algebra](https://allendowney.github.io/ThinkLinearAlgebra/index.html)

Copyright 2025 [Allen B. Downey](https://allendowney.com)

Code license: [MIT License](https://mit-license.org/)

Text license: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/)